In [31]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import random_split, DataLoader
from torchvision import datasets, transforms
import pathlib
from torch.amp import GradScaler, autocast
import time
import math

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Enhanced Data Augmentation
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Dataset
training_dir = pathlib.Path('/kaggle/input/radar-signal-classification/training_set')
full_dataset = datasets.ImageFolder(root=str(training_dir), transform=transform)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

class_names = full_dataset.classes
num_classes = len(class_names)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, pin_memory=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, pin_memory=True, num_workers=4)

print(f"Classes: {class_names}")
print(f"Training samples: {train_size}, Validation samples: {val_size}")

# Optimized CNN Model
class RadarSignalCNN(nn.Module):
    def __init__(self, num_classes=8):
        super(RadarSignalCNN, self).__init__()
        
        # Feature extraction
        self.features = nn.Sequential(
            # Block 1
            nn.Conv2d(3, 32, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 32, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.2),
            
            # Block 2
            nn.Conv2d(32, 64, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.3),
            
            # Block 3
            nn.Conv2d(64, 120, kernel_size=3, padding=1, bias=False),  # Giảm từ 128 xuống 120
            nn.BatchNorm2d(120),
            nn.ReLU(inplace=True),
            nn.Conv2d(120, 120, kernel_size=3, padding=1, bias=False),  # Giảm từ 128 xuống 120
            nn.BatchNorm2d(120),
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d((1, 1))
        )
        
        # Classifier
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(120, 120),  # Giảm từ 128 xuống 120
            nn.BatchNorm1d(120),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(120, num_classes)
        )
        
        # Initialize weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.BatchNorm1d)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return F.log_softmax(x, dim=1)

# Model initialization
model = RadarSignalCNN(num_classes=num_classes).to(device)

# Count parameters
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"🔢 Total trainable parameters: {total_params}")

# Enhanced training configuration
criterion = nn.CrossEntropyLoss(label_smoothing=0.1).to(device)
optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.001, 
                                        steps_per_epoch=len(train_loader), 
                                        epochs=50, 
                                        anneal_strategy='cos')
scaler = GradScaler('cuda')

# Training function without early stopping
def train(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=50):
    best_val_acc = 0.0
    
    for epoch in range(epochs):
        start_time = time.time()
        model.train()
        train_loss = 0.0
        correct = 0
        total = 0
        
        for images, labels in train_loader:
            images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            optimizer.zero_grad(set_to_none=True)
            
            with autocast(device_type='cuda'):
                outputs = model(images)
                loss = criterion(outputs, labels)
            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            scheduler.step()
            
            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
        
        train_acc = 100. * correct / total
        
        # Validation
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                with autocast(device_type='cuda'):
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                
                val_loss += loss.item()
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()
        
        val_acc = 100. * correct / total
        
        # Save best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), "best_model.pt")
        
        epoch_time = time.time() - start_time
        current_lr = optimizer.param_groups[0]['lr']
        
        print(f"Epoch {epoch+1:2d} | "
              f"Train Loss: {train_loss/len(train_loader):.4f} | "
              f"Train Acc: {train_acc:.2f}% | "
              f"Val Loss: {val_loss/len(val_loader):.4f} | "
              f"Val Acc: {val_acc:.2f}% | "
              f"Time: {epoch_time:.2f}s | "
              f"LR: {current_lr:.6f}")
    
    return model

# Train the model
torch.cuda.empty_cache()
model = train(model, train_loader, val_loader, criterion, optimizer, scheduler)

# Save the model
example_input = torch.randn(1, 3, 128, 128).to(device)
traced_model = torch.jit.trace(model, example_input)
traced_model.save("22119129_22119067.pt")
print("✅ Model saved as 22119129_22119067.pt")

torch.cuda.empty_cache()

Using device: cuda
Classes: ['B-FM', 'Barker', 'CPFSK', 'DSB-AM', 'GFSK', 'LFM', 'Rect', 'SSB-AM']
Training samples: 5120, Validation samples: 1280
🔢 Total trainable parameters: 280688
Epoch  1 | Train Loss: 1.9511 | Train Acc: 27.75% | Val Loss: 1.8272 | Val Acc: 26.88% | Time: 8.09s | LR: 0.000051
Epoch  2 | Train Loss: 1.7475 | Train Acc: 33.95% | Val Loss: 1.7462 | Val Acc: 30.94% | Time: 8.06s | LR: 0.000082
Epoch  3 | Train Loss: 1.6504 | Train Acc: 38.07% | Val Loss: 1.6962 | Val Acc: 31.41% | Time: 7.94s | LR: 0.000132
Epoch  4 | Train Loss: 1.5551 | Train Acc: 44.08% | Val Loss: 1.6466 | Val Acc: 36.25% | Time: 8.38s | LR: 0.000199
Epoch  5 | Train Loss: 1.4629 | Train Acc: 49.14% | Val Loss: 1.6357 | Val Acc: 43.28% | Time: 7.97s | LR: 0.000280
Epoch  6 | Train Loss: 1.3412 | Train Acc: 54.69% | Val Loss: 1.4564 | Val Acc: 47.89% | Time: 7.93s | LR: 0.000372
Epoch  7 | Train Loss: 1.1982 | Train Acc: 63.11% | Val Loss: 1.3804 | Val Acc: 49.14% | Time: 7.91s | LR: 0.000470
Epo